In [20]:
import { ChatOpenAI } from "@langchain/openai";

const llm = new ChatOpenAI({ model: "gpt-4o-mini" }).bindTools([
  { type: "web_search_preview" },
]);

// await llm.invoke("What was a positive news story from today?");

In [29]:
import fs from "node:fs";
import OpenAI from "../../../node_modules/openai/index.js";
const openai = new OpenAI();

function guessExtFromContentType(contentType) {
  if (!contentType) return "bin";
  const ct = contentType.toLowerCase();
  if (ct.includes("pdf")) return "pdf";
  if (ct.includes("html")) return "html";
  if (ct.includes("markdown")) return "md";
  if (ct.includes("plain")) return "txt";
  if (ct.includes("json")) return "json";
  return "bin";
}

async function createFile(filePath) {
  let result;
  if (filePath.startsWith("http://") || filePath.startsWith("https://")) {
    // Descargar contenido desde URL y asignar nombre/tipo válidos
    const res = await fetch(filePath);
    const buffer = await res.arrayBuffer();

    const contentType = res.headers.get("content-type") || "";
    const urlObj = new URL(filePath);
    let baseName = urlObj.pathname.split("/").filter(Boolean).pop() || "download";

    // Normalizar nombre y extensión
    const hasExt = baseName.includes(".");
    const ext = hasExt ? baseName.split(".").pop().toLowerCase() : guessExtFromContentType(contentType);
    if (!hasExt) baseName = `${baseName}.${ext}`;

    const file = new File([buffer], baseName, { type: contentType || undefined });

    result = await openai.files.create({
      file,
      purpose: "assistants",
    });
  } else {
    // Archivo local
    const fileContent = fs.createReadStream(filePath);
    result = await openai.files.create({
      file: fileContent,
      purpose: "assistants",
    });
  }
  return result.id;
}

// Reemplaza con tu ruta o URL
const fileId = await createFile(
  "https://cdn.openai.com/API/docs/deep_research_blog.pdf"
);

console.log(fileId);

file-Brxbnpfee1AtFkTpmmTMv3


In [30]:
const vectorStore = await openai.vectorStores.create({
    name: "deepresearch_base",
});
console.log(vectorStore.id);

vs_68af545f954c8191907af81273300de8


In [31]:
const addFileToVectorStore = async (vectorStore:any , fileId:string) => {
    await openai.vectorStores.files.create(
        vectorStore.id,
        {
            file_id: fileId,
        }
    );
}

In [32]:
await addFileToVectorStore(vectorStore, fileId);

Comprobar estado
Ejecute este código hasta que el archivo esté listo para ser utilizado (es decir, cuando el estado sea completed).

In [ ]:
const result = await openai.vectorStores.files.list(vectorStore.id);
console.log(result);

In [33]:
import { ChatOpenAI } from "@langchain/openai";

const llm = new ChatOpenAI({ model: "gpt-4o-mini" }).bindTools([
  { type: "file_search", vector_store_ids: [vectorStore.id] },
]);

await llm.invoke("cuanto toma una busqueda profunda");

AIMessage {
  "id": "msg_68af54ad689c819990c12e212e7415620c40003e15713379",
  "content": [
    {
      "type": "text",
      "text": "Una búsqueda profunda puede tardar entre 5 a 30 minutos en completarse, dependiendo de la complejidad de la consulta y la cantidad de información que se necesite recopilar.",
      "annotations": [
        "[Object]"
      ]
    }
  ],
  "additional_kwargs": {
    "tool_outputs": [
      {
        "id": "fs_68af54aa68d0819993172ab08677613d0c40003e15713379",
        "type": "file_search_call",
        "status": "completed",
        "queries": "[Array]",
        "results": null
      }
    ]
  },
  "response_metadata": {
    "id": "resp_68af54a96d8081998252140154166fb50c40003e15713379",
    "estimatedTokenUsage": {
      "promptTokens": 17340,
      "completionTokens": 67,
      "totalTokens": 17407
    },
    "model": "gpt-4o-mini-2024-07-18",
    "created_at": 1756320937,
    "incomplete_details": null,
    "metadata": {},
    "object": "response",
    "